In [1]:
# file:///C:/Users/Admin/AppData/Local/Google/Chrome/User%20Data/Default/
# go to this link in browser if want to check internal info of google browser

#http://bar-navig.yandex.ru/u?show=31&url=https://www.coursera.org
#on this lick we get xml response about web_site

#https://www.kakprosto.ru/kak-109671-kak-uznat-tematiku-sayta
#on this lick we can get insructions to parse site data

In [2]:
import os
import shutil
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import re


data_path = os.path.expanduser('~')+"\\AppData\\Local\\Google\\Chrome\\User Data\\Default"

srcfile = data_path+'\\History'

try:
    dstroot = 'chrome_history'
    dstdir =  os.path.join(dstroot)
    os.makedirs(dstdir) 
except:
    print ('dir already here ')


shutil.copy(srcfile, dstdir)

data_path = dstdir+'\\History'




dir already here 


In [3]:
select_statement = "SELECT * FROM urls, visits WHERE urls.id = visits.url;"
select_statement = "select datetime(last_visit_time/1000000-11644473600,'unixepoch','localtime') as dt_last_visit, * FROM urls ORDER BY last_visit_time DESC"

select_statement = ''.join([
                        'SELECT ' , 'v.id , v.url as url_id , v.visit_time , v.visit_duration', ' \n'
                                  ," , datetime(v.visit_time/1000000-11644473600,'unixepoch','localtime') as dt_visit_time" , ' \n'
                                  , ' , u.url , u.title , u.visit_count , u.typed_count , ' , ' \n'
                                  , "datetime(u.last_visit_time/1000000-11644473600,'unixepoch','localtime') as dt_last_visit" , ' \n'
                        ,'from ' , 'visits v' , ' \n'
                        ,'left join ' , 'urls u on v.url = u.id  ' , ' \n'
                        ,'order by ' , ' 2 desc'
                                        
                        ])



print(select_statement)

SELECT v.id , v.url as url_id , v.visit_time , v.visit_duration 
 , datetime(v.visit_time/1000000-11644473600,'unixepoch','localtime') as dt_visit_time 
 , u.url , u.title , u.visit_count , u.typed_count ,  
datetime(u.last_visit_time/1000000-11644473600,'unixepoch','localtime') as dt_last_visit 
from visits v 
left join urls u on v.url = u.id   
order by  2 desc


In [4]:
c = sqlite3.connect(data_path)
cursor = c.cursor()



cursor.execute(select_statement)


df_chrome_history = pd.read_sql_query(select_statement, c)

df_chrome_history['dt_last_visit'] =  [ datetime.strptime( x , '%Y-%m-%d %H:%M:%S') for x in df_chrome_history['dt_last_visit'] ]
df_chrome_history['dt_visit_time'] =  [ datetime.strptime( x , '%Y-%m-%d %H:%M:%S') for x in df_chrome_history['dt_visit_time'] ]
df_chrome_history.drop(['visit_time'], axis=1 , inplace = True)

#очистим данные от мусора
documents = [re.split('/', x)[2] for x in  df_chrome_history['url']]
df_chrome_history['documents'] = documents
c.close()

#очистим от мусорных сайтов
trash = ['mail.google.com' , 'docs.google.com' , 'e.mail.ru' , 'gmail.com' , 'mail.ru']

df_chrome_history = df_chrome_history[(df_chrome_history.documents != 'localhost:8888')
                                     &(df_chrome_history.title != '')
                                     &~(df_chrome_history.documents.isin(trash))
                                     ][:]

In [5]:
#посмотрим топ сайтов за последнее время
df_chrome_history[(df_chrome_history.dt_last_visit > datetime.now() - timedelta(days=30))].groupby(['documents'])[['id']].count().sort_values('id' , ascending = False)

,id
documents,
www.youtube.com,337
www.google.ru,287
www.coursera.org,267
drive.google.com,239
github.com,223
www.avito.ru,169
vk.com,125
ru.aliexpress.com,125
basecamp.com,107


In [6]:

#если хотим посмотреть что-то определенное
interest = []
for i in df_chrome_history['documents'].unique():
    if 'github' in i:
        interest.append(i)
        
        
df_chrome_history = df_chrome_history[(df_chrome_history.documents.isin(interest))
                                      &(df_chrome_history.dt_last_visit > datetime.now() - timedelta(days=8))
                                     ][:]

In [7]:
df_chrome_history.groupby(['documents'])[['id']].count().sort_values(['id'] , ascending = False)
df_chrome_history[(df_chrome_history.dt_last_visit > pd.to_datetime('20180301', format='%Y%m%d', errors='ignore'))]

,id,url_id,visit_duration,dt_visit_time,url,title,visit_count,typed_count,dt_last_visit,documents
188,50036,21842,0,2018-03-30 02:20:08,https://github.com/scikit-learn/scikit-learn/b...,scikit-learn/dbscan_.py at master · scikit-lea...,1,0,2018-03-30 02:20:08,github.com
214,49958,21816,482131771,2018-03-30 01:23:18,https://github.com/mwaskom/seaborn/issues/1092,one kdeplot function error of seaborn · Issue ...,2,0,2018-03-30 01:23:18,github.com
215,49956,21816,0,2018-03-30 01:23:15,https://github.com/mwaskom/seaborn/issues/1092,one kdeplot function error of seaborn · Issue ...,2,0,2018-03-30 01:23:18,github.com
216,49957,21815,0,2018-03-30 01:23:17,https://github.com/mwaskom/seaborn/issues/1103,one kdeplot function error of seaborn · Issue ...,3,0,2018-03-30 01:23:17,github.com
217,49955,21815,0,2018-03-30 01:23:15,https://github.com/mwaskom/seaborn/issues/1103,one kdeplot function error of seaborn · Issue ...,3,0,2018-03-30 01:23:17,github.com
218,49954,21815,0,2018-03-30 01:23:06,https://github.com/mwaskom/seaborn/issues/1103,one kdeplot function error of seaborn · Issue ...,3,0,2018-03-30 01:23:17,github.com
223,49948,21810,0,2018-03-30 01:19:03,https://gist.github.com/auth/github/callback?r...,How to plot nice 2d density plots of samples i...,1,0,2018-03-30 01:19:03,gist.github.com
224,49947,21809,0,2018-03-30 01:19:03,https://github.com/login/oauth/authorize?clien...,How to plot nice 2d density plots of samples i...,1,0,2018-03-30 01:19:03,github.com
225,49946,21808,0,2018-03-30 01:19:03,https://gist.github.com/auth/github?return_to=...,How to plot nice 2d density plots of samples i...,1,0,2018-03-30 01:19:03,gist.github.com
226,49949,21807,8241217,2018-03-30 01:19:03,https://gist.github.com/dfm/2008724,How to plot nice 2d density plots of samples i...,2,0,2018-03-30 01:19:03,gist.github.com


In [8]:
import re

punctuation_pattern = ' |\.$|\. |, |\/|\(|\)|\'|\"|\!|\?|\+|\:'
documents = [re.split('/', x)[2] for x in  df_chrome_history['url']]
text = [re.split(punctuation_pattern, x.lower()) for x in  df_chrome_history['title']]

In [9]:
%%time
#стеммизируем слова

import time
start = time.time()
import pymystem3
mystem = pymystem3 . Mystem ( )

r = 0
for row in text:
    row_new = []
    for word in row:
        word = word.encode('utf8','ignore').decode()
        if len(word) > 3 and '\\u' != word[:2] and '<' != word[:1] and '\\' != word[:1]:
            
            #test
            if r <= 1:
                new_word = mystem.lemmatize (word)[0]
            else:
                new_word = word
            row_new.append(new_word)
        
    
    text[r] = row_new
    r += 1
    

    
    
stop = time.time()
print (u"Время, затраченное lemmatize- %f на обработку  "%(stop - start))

Время, затраченное lemmatize- 16.607476 на обработку  
Wall time: 16.6 s


# bigartm

In [11]:
#vowpalrabbit
import codecs
history_vowpalrabbit_all = ''

text_file = codecs.open("chrome_history_vowpalrabbit.txt", "w", "utf-8")
errors = 0
for doc, row in zip(documents,text):
    row_str = ' '.join(row)
    seq = (doc , '|' , 'text ' , row_str , ' \n')
    history_vowpalrabbit = ''.join(seq)
    try:
        text_file.write(history_vowpalrabbit)
    except:
        errors += 1
        continue
    
    
text_file.close()

In [12]:
import artm
folder = 'chrome_history_bathces'
data = "chrome_history_vowpalrabbit.txt"

batch_vectorizer = artm.BatchVectorizer(data_path=data, data_format="vowpal_wabbit", target_folder=folder, 
                                       batch_size=100)

In [13]:
dictionary = artm.Dictionary(data_path=folder)# загрузка данных в словарь

In [14]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
model = artm.ARTM(num_topics=3,
                  num_document_passes=10,#10 проходов по документу
                  dictionary=dictionary,
                  scores=[artm.TopTokensScore(name='top_tokens_score')])

In [15]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)#10 проходов по коллекции

In [16]:
top_tokens = model.score_tracker['top_tokens_score']

In [17]:
for topic_name in model.topic_names:
    print (topic_name)
    for (token, weight) in zip(top_tokens.last_tokens[topic_name],
                               top_tokens.last_weights[topic_name]):    
         print (token, '-', round(weight,3))

topic_0
seaborn - 0.222
mwaskom - 0.111
error - 0.111
kdeplot - 0.111
issue - 0.111
function - 0.111
#1092 - 0.089
scikit - 0.067
# - 0.022
master - 0.022
topic_1
byte - 0.143
start - 0.071
invalid - 0.071
0xf6 - 0.071
#651 - 0.071
decode - 0.071
codec - 0.071
utf-8 - 0.071
unicodedecodeerror - 0.071
position - 0.071
topic_2
python - 0.167
samples - 0.167
plots - 0.167
nice - 0.167
plot - 0.167
density - 0.167


In [19]:
#из топика найдем новость в интернете
top_tokens.last_tokens['topic_0'][0]

'seaborn'

In [256]:
#новостной сайт - https://newsapi.org/docs/client-libraries/python - ищет только в цензорных
#новострной сайт 2 - https://webhose.io/web-content-api - крутой везде ище

import requests
import json
from bs4 import BeautifulSoup
#from newsapi import NewsApiClient

newsapi_key = 'f45b069a9ab844369005986b34bb6764'
webhose_key = '96b16ce1-f430-45fb-937f-14c395579994'
text = 'http://webhose.io/filterWebContent?token='+webhose_key+'&format=json&ts=1520713935519&sort=crawled&q=sportage%20language%3Arussian%20site%3Adrom.ru'
r = requests.get(text, auth=('user', 'pass'))


In [257]:
response = r.text

In [258]:
json_response = json.loads(response)

In [259]:
json_response

{'moreResultsAvailable': 0,
 'next': '/filterWebContent?token=96b16ce1-f430-45fb-937f-14c395579994&format=json&ts=1521672610018&q=sportage+language%3Arussian+site%3Adrom.ru&sort=crawled',
 'posts': [{'author': '',
   'crawled': '2018-03-22T00:50:10.018+02:00',
   'entities': {'locations': [], 'organizations': [], 'persons': []},
   'external_links': [],
   'highlightText': '',
   'highlightTitle': '',
   'language': 'russian',
   'ord_in_thread': 0,
   'published': '2018-03-21T17:22:00.000+02:00',
   'rating': None,
   'text': 'Не пройдет любой автомобиль). Вы все врете). Только корейцы и японцы. После того, как даже в России люди стали понимать, что цены на Хайлендер и Прадо- совсем не равны их оснащаемости и качеству- продажи падают который месяц подряд. Но, что делать, надо же ваньке русскому впаривать Субару за 2500, который стоит 1млн 500. И мы достаем из кармана надежность, и Тойота за 5 американских зарплат, становится надежным бизнес автомобилем Камри, а не сел=поехал у лати См

C:\Users\Admin\Anaconda3\lib\site-packages\bs4\element.py:15: DeprecationWarning: invalid escape sequence \s
  whitespace_re = re.compile("\s+")
C:\Users\Admin\Anaconda3\lib\site-packages\bs4\element.py:1407: DeprecationWarning: invalid escape sequence \d
  pseudo_attributes = re.match('([a-zA-Z\d-]+)\(([a-zA-Z\d]+)\)', pseudo)
C:\Users\Admin\Anaconda3\lib\site-packages\bs4\dammit.py:49: DeprecationWarning: invalid escape sequence \?
  '^<\?.*encoding=[\'"](.*?)[\'"].*\?>'.encode(), re.I)
C:\Users\Admin\Anaconda3\lib\site-packages\lxml\_elementpath.py:63: DeprecationWarning: invalid escape sequence \.
  "\.\.|"
C:\Users\Admin\Anaconda3\lib\site-packages\lxml\_elementpath.py:64: DeprecationWarning: invalid escape sequence \(
  "\(\)|"
C:\Users\Admin\Anaconda3\lib\site-packages\lxml\_elementpath.py:65: DeprecationWarning: invalid escape sequence \[
  "[/.*:\[\]\(\)@=])|"
C:\Users\Admin\Anaconda3\lib\site-packages\lxml\_elementpath.py:66: DeprecationWarning: invalid escape sequence \{
  "